# Ініціалізація проекту

In [1]:
!pip install urllib
!pip install requests
!pip install fuzzywuzzy
!pip install python-Levenshtein
# !pip install SpeechRecognition
!pip install pyttsx3
!pip install gtts

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
     |████████████████████████████████| 50 kB 2.7 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149856 sha256=843caa3b49f8cfbd0a74fea047ab4828d93f9fc8a407cddb8cf05e6a79fdaaed
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Study/DevNet/HikkaShop']

In [4]:
sys.path.append("/content/drive/MyDrive/Study/DevNet/HikkaShop")

# Модуль обробки природної мови (NLP)

## Функція отримання відповіді від WaifuAI Api

In [2]:
import requests as re
import urllib.parse
import Keys

def get_response(from_name, to_name, situation: list, user_id, message):
  data_dict = {
    "from_name": from_name,
    "to_name": to_name,
    "situation": ' '.join(map(str, situation)),
    "message": message,

    "user_id": user_id,
    "preset_mode": "waifu",
    "translate_from": "uk",
  }

  params = {
    "url": "https://waifu-ai.p.rapidapi.com/path",
    "headers": {
      'content-type': 'application/x-www-form-urlencoded',
      'x-rapidapi-host': 'waifu-ai.p.rapidapi.com',
      'x-rapidapi-key': Keys.keyMax
    },
    "data": "&".join(param+"="+urllib.parse.quote(value) for param, value in data_dict.items())
  }
  
  response = re.post(**params)
  return response.text, response

## Створення логфайлу бесіди

In [3]:
def logOutput(log):
  from datetime import datetime
  now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
  with open("logfiles/"+now+".txt", "a", encoding="UTF-8") as ouf:
    ouf.write(log)

## Суперклас WaifuAI

In [4]:
class WaifuAI():
  @staticmethod
  def setattrs(base_situation: str, user_id: str = None):
    WaifuAI.base_situation = base_situation
    WaifuAI.situation = [base_situation]
    
    user_id = None if user_id == "" else user_id
    WaifuAI.user_id = user_id or WaifuAI.generate_user_id()
  
  @staticmethod
  def append_situation(situation: str):
    WaifuAI.situation.append(situation)

  @staticmethod
  def reset_situation():
    WaifuAI.situation = [WaifuAI.base_situation]
  
  def generate_user_id():
    import string
    import random
    alnum = string.ascii_letters + string.digits
    user_id = ''.join(random.choice(alnum) for i in range(64))
    print(user_id)
    return user_id

## Клас продавця

In [5]:
class Seller(WaifuAI):
  phrases = []
  
  def __init__(self, name: str, stock: dict):
    self.name = name
    self.stock = stock
    self.sync_stock()

  def sync_stock(self):
    for merch, v in self.stock.items():
      qnty = v[0]
      price = v[1]
      if qnty != 0:
        super().append_situation(f"{self.name} може продати {merch} у кількості {qnty} за ціною {price} гривень.")
      else:
        super().append_situation(f"{merch} закінчився, тому {self.name} не може продати {merch}.")
        
  def get_order(self, message, tolerance=70):
    import re
    from fuzzywuzzy import fuzz

    match_obj = re.match("купити ([\w\s]*)\.", message)
    if match_obj:
      finded_merch = match_obj.groups()[0]

      max_ratio = 0
      merch = None
      for stock_merch in self.stock.keys():
        ratio = fuzz.ratio(finded_merch, stock_merch)

        if ratio > max_ratio or max_ratio == 0:
          max_ratio = ratio
          merch = stock_merch

      if max_ratio >= tolerance and merch is not None:
        return merch
      return "Not Found"
    return None

## Клас покупця

In [6]:
class Customer(WaifuAI):
  phrases = []

  def __init__(self, name: str, cash: float):
    self.name = name
    self.cash = cash
    self.sync_cash()

  def sync_cash(self):
    super().append_situation(f"{self.name} має при собі {self.cash} гривень.")

  def purchase(self, seller: Seller, merch):
    if merch == "Not Found":
      super().situation.append(f"{self.name} намагався купити товар, але {seller.name} не зрозуміла що це за товар.")
      return

    merch_qnty = seller.stock[merch][0]
    merch_price = seller.stock[merch][1]
    
    if merch_qnty == 0:
      super().append_situation(f"{self.name} намагався купити {merch}, але {merch} закінчився.")
      return

    if self.cash < merch_price:
      super().append_situation(f"{self.name} намагався купити {merch}, але {self.name} не має стільки грошей.")
      return

    seller.stock[merch][0] -= 1
    self.cash -= merch_price
    super().reset_situation()
    self.sync_cash()
    seller.sync_stock()
    print("---")
    print(f"Каса-сама: Баланс {self.name} становить {self.cash}.")
    print("---")

## Вхідні дані розмови

In [7]:
from merch_list import merch_list

#@title Ствоерення діючих осіб{ run: "auto", form-width: "50%" }
#@markdown Покупець
customer_name = "Тарас-кун" #@param {type: "raw"}
cash          = 12.3#@param {type:"number"}

#@markdown Продавець та система
seller_name = "Акено-тян" #@param {type: "raw"}
situation   = "Акено-тян продає аніме-товари в магазині." #@param {type: "raw"}

## Надсилання повідомлень

In [8]:
WaifuAI.setattrs(situation, WaifuAI.generate_user_id())

customer = Customer(customer_name, cash)
seller = Seller(seller_name, merch_list)

log = f"""from_name = {customer.name}
to_name   = {seller.name}
situation = {WaifuAI.situation}
user_id   = {WaifuAI.user_id}
---\n
"""

print("Каса-сама: Для замовлення товару наберіть 'купити НАЗВА_ТОВАРУ.'")
print("Каса-сама: Для завершення бесіди наберіть 'Пока!'")
print("---")

while(True):
  message = input(f"{customer.name}: ")
  log += f"{customer.name}: {message}\n"

  customer.phrases.append(message)

  merch = seller.get_order(message)
  if merch is not None:
    customer.purchase(seller, merch)

  response_text, response = get_response(
    from_name = customer.name,
    to_name   = seller.name,
    situation = WaifuAI.situation,
    user_id   = WaifuAI.user_id,
    message   = message
  )

  seller.phrases.append(response_text)

  sellerResp = f'{seller.name}: {response_text}'
  print(sellerResp)
  log += sellerResp+"\n"

  if (message == "Пока!"):
    logOutput(log)
    break

dPdLqtAjoDEgzpB42mWZCiFhWeiAicOzKJakODQAwvTCWVqOfGHvtb4jWLVNcgFO
Каса-сама: Для замовлення товару наберіть 'купити НАЗВА_ТОВАРУ.'
Каса-сама: Для завершення бесіди наберіть 'Пока!'
---
Акено-тян: Я тут, щоб допомогти вам.
Акено-тян: Так, у мене є кілька друзів, яким потрібна ваша допомога. Якщо зможу я заплатити вам винагороду і подарувати подарунки як вибачення за незручності...
Акено-тян: Дакіма... що? ДАКІМА-КУРУ - кумедна пухнаста художня робота дівчини в одязі з хутра або шкіри/чоботи тощо, зазвичай на різних етапах роздягання або сексуальних актів з її участю, часто включаючи бондаж/садомазохізм (SM), фетиші, такі як латексний фетишизм і цнотливість/шиття, але також включає багато інших тем, як-от інцест (
Акено-тян: В порядку! Тож тепер приступимо до справи. Ми знаємо, що ви шукаєте певну людину на ім’я «Незламний». На жаль, її схопила інша авантюристка на ім’я Фландр Скарлет, яка володіє магією, яку неможливо описати, і чиї сили конкурують навіть із моїми. Зараз вона живе в цьом

TypeError: logOutput() got an unexpected keyword argument 'filedir'

## Пояснення проблеми

![](explaining_problem.jpg)

# Модуль синтезу мовлення (Speech Synthesis)

In [10]:
import pyttsx3
engine = pyttsx3.init()

import gtts
from playsound import playsound

for i, cust_phrase in enumerate(customer.phrases):
  text = cust_phrase.replace("и", "ы").replace("і", "и")
  engine.say(text)
  engine.runAndWait()

  tts = gtts.gTTS(seller.phrases[i], lang="ja")
  tts.save("speechOutput/seller"+str(i)+".mp3")
  playsound("speechOutput/seller"+str(i)+".mp3")